In [1]:
import pandas as pd
import json
from datetime import datetime
import numpy as np

In [2]:
# to read in json file
with open('professionals_nested.json', 'r') as f:
    data = json.load(f)
professionals = data['professionals']

### EDA

In [3]:
print(f"Number of professionals in the dataset: {len(professionals)}")

Number of professionals in the dataset: 10


In [4]:
professionals_df = pd.DataFrame([{
    'professional_id': p['professional_id'],
    'years_experience': p['years_experience'],
    'current_industry': p['current_industry'],
    'current_role': p['current_role'],
    'education_level': p['education_level'],
    'num_jobs': len(p['jobs']),
    'num_skills': len(p['skills']),
    'num_certifications': len(p.get('certifications', [])),
    'num_education': len(p['education'])
} for p in professionals])
print(professionals_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   professional_id     10 non-null     object
 1   years_experience    10 non-null     int64 
 2   current_industry    10 non-null     object
 3   current_role        10 non-null     object
 4   education_level     10 non-null     object
 5   num_jobs            10 non-null     int64 
 6   num_skills          10 non-null     int64 
 7   num_certifications  10 non-null     int64 
 8   num_education       10 non-null     int64 
dtypes: int64(5), object(4)
memory usage: 848.0+ bytes
None


In [5]:
print(professionals_df.describe())

       years_experience   num_jobs  num_skills  num_certifications  \
count         10.000000  10.000000   10.000000            10.00000   
mean           9.200000   3.200000    3.400000             1.90000   
std            4.732864   0.918937    0.699206             1.37032   
min            3.000000   2.000000    3.000000             0.00000   
25%            6.250000   3.000000    3.000000             1.00000   
50%            8.500000   3.000000    3.000000             1.50000   
75%           11.500000   3.750000    3.750000             2.75000   
max           18.000000   5.000000    5.000000             4.00000   

       num_education  
count      10.000000  
mean        1.600000  
std         0.699206  
min         1.000000  
25%         1.000000  
50%         1.500000  
75%         2.000000  
max         3.000000  


In [6]:
print(professionals_df['current_industry'].value_counts())

current_industry
Technology    4
Marketing     2
Finance       2
Healthcare    1
Retail        1
Name: count, dtype: int64


In [7]:
print(professionals_df['education_level'].value_counts())

education_level
Bachelors    5
Masters      3
PhD          1
MBA          1
Name: count, dtype: int64


In [8]:
print("\nNested Data Structures:")
print(f"Job fields: {list(professionals[0]['jobs'][0].keys())}")
print(f"Skill fields: {list(professionals[0]['skills'][0].keys())}")
print(f"Education fields: {list(professionals[0]['education'][0].keys())}")


Nested Data Structures:
Job fields: ['job_id', 'company', 'industry', 'role', 'start_date', 'end_date', 'salary_band']
Skill fields: ['skill_id', 'skill_name', 'proficiency_level', 'years_experience']
Education fields: ['education_id', 'degree', 'institution', 'field_of_study', 'graduation_date']


### Create DataFrames from nested structure

In [9]:
professionals_df = pd.DataFrame([{
    'professional_id': p['professional_id'],
    'years_experience': p['years_experience'],
    'current_industry': p['current_industry'],
    'current_role': p['current_role'],
    'education_level': p['education_level']
} for p in professionals])

In [10]:
jobs_list = []
for p in professionals:
    for job in p['jobs']:
        job_dict = job.copy()
        job_dict['professional_id'] = p['professional_id']
        jobs_list.append(job_dict)
        
jobs_df = pd.DataFrame(jobs_list)

In [11]:
skills_list = []
for p in professionals:
    for skill in p['skills']:
        skill_dict = skill.copy()
        skill_dict['professional_id'] = p['professional_id']
        skills_list.append(skill_dict)
        
skills_df = pd.DataFrame(skills_list)

In [12]:
certs_list = []
for p in professionals:
    for cert in p.get('certifications', []):
        cert_dict = cert.copy()
        cert_dict['professional_id'] = p['professional_id']
        certs_list.append(cert_dict)
        
certs_df = pd.DataFrame(certs_list)

In [13]:
education_list = []
for p in professionals:
    for edu in p['education']:
        edu_dict = edu.copy()
        edu_dict['professional_id'] = p['professional_id']
        education_list.append(edu_dict)
        
education_df = pd.DataFrame(education_list)

### Data Validation and QC
- Validation of overlapping jobs
- Identification of expired certifications
- Outliers in years of experience
- Validation of date formats and NULLs

In [14]:
def convert_date(date_str):
    if pd.isnull(date_str) or date_str is None:
        return None
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except (ValueError, TypeError):
        return None

jobs_df['start_date'] = jobs_df['start_date'].apply(convert_date)
jobs_df['end_date'] = jobs_df['end_date'].apply(convert_date)

In [ ]:
def validate_job_dates(prof_id):
    prof_jobs = jobs_df[jobs_df['professional_id'] == prof_id].sort_values('start_date')
    
    prev_end = None
    issues = []
    
    for i, job in prof_jobs.iterrows():
        if prev_end and job['start_date'] and prev_end > job['start_date']:
            issues.append(f"Job {job['job_id']} starts before previous job ends")
        
        if job['end_date'] and job['start_date'] and job['end_date'] < job['start_date']:
            issues.append(f"Job {job['job_id']} ends before it starts")
            
        prev_end = job['end_date']
    
    return issues

job_date_issues = {prof_id: validate_job_dates(prof_id) for prof_id in professionals_df['professional_id']}

In [16]:
job_date_issues = {prof_id: validate_job_dates(prof_id) for prof_id in professionals_df['professional_id']}

certs_df['date_earned'] = certs_df['date_earned'].apply(convert_date)
certs_df['expiration_date'] = certs_df['expiration_date'].apply(convert_date)

now = datetime.now()
expired_certs = certs_df[certs_df['expiration_date'] < now]

In [17]:
def detect_outliers(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

experience_outliers = detect_outliers(professionals_df, 'years_experience')
skill_experience_outliers = detect_outliers(skills_df, 'years_experience')

### Creating a dimensional model
star schema was used to create ta dimensional model
dimension tables include following: professionals (core professional attributes), skills, certificates, education, time, and industry

fact table includes following: jobs, and career progression

In [18]:
def create_time_dim():
    all_dates = []
    #job dates
    all_dates.extend(jobs_df['start_date'].dropna().tolist())
    all_dates.extend(jobs_df['end_date'].dropna().tolist())
    #certificate dates
    all_dates.extend(certs_df['date_earned'].dropna().tolist())
    all_dates.extend(certs_df['expiration_date'].dropna().tolist())
    all_dates.extend([convert_date(date) for date in education_df['graduation_date'].dropna()])
    
    unique_dates = list(set(all_dates))
    unique_dates.sort()
    #time demension
    time_dim = pd.DataFrame({
        'date_id': range(len(unique_dates)),'full_date': unique_dates,'year': [d.year for d in unique_dates],
        'month': [d.month for d in unique_dates],
        'quarter': [((d.month-1)//3)+1 for d in unique_dates]
    })
    
    return time_dim

time_dim = create_time_dim()

In [19]:
industry_dim = pd.DataFrame({
    'industry_id': range(len(jobs_df['industry'].unique())),
    'industry_name': jobs_df['industry'].unique()
})

In [20]:
def career_prog_fact():
    #job duration
    jobs_df['duration_months'] = None
    for i, row in jobs_df.iterrows():
        if row['start_date'] and row['end_date']:
            months = (row['end_date'].year - row['start_date'].year) * 12 + (row['end_date'].month - row['start_date'].month)
            jobs_df.at[i, 'duration_months'] = months
        elif row['start_date'] and not row['end_date']:
            months = (now.year - row['start_date'].year) * 12 + (now.month - row['start_date'].month)
            jobs_df.at[i, 'duration_months'] = months
    
    prog_data = []
    
    for prof_id in professionals_df['professional_id']:
        prof_jobs = jobs_df[jobs_df['professional_id'] == prof_id].sort_values('start_date')
        
        if len(prof_jobs) > 1:
            #salary growth
            sb1 = prof_jobs.iloc[0]['salary_band']
            sb2 = prof_jobs.iloc[-1]['salary_band']
            salary_growth = sb2 - sb1
            #current position
            j1 = prof_jobs.iloc[0]['start_date']
            j2 = prof_jobs.iloc[-1]['start_date']
            
            if j1 and j2:
                months_to_current = (j2.year - j1.year) * 12 + (j2.month - j1.month)
            else:
                months_to_current = None
            # industry changes
            industry_changes = len(prof_jobs['industry'].unique()) - 1
            num_roles = len(prof_jobs)
            prog_data.append({
                'professional_id': prof_id,
                'num_roles': num_roles,
                'salary_band_growth': salary_growth,
                'months_to_current_role': months_to_current,
                'industry_changes': industry_changes,
                'avg_job_duration': prof_jobs['duration_months'].mean()
            })
    return pd.DataFrame(prog_data)
career_prog_fact = career_prog_fact()

In [21]:
professionals_df.to_csv('output/dim_professionals.csv', index=False)
jobs_df.to_csv('output/fact_jobs.csv', index=False)
skills_df.to_csv('output/dim_skills.csv', index=False)
certs_df.to_csv('output/dim_certifications.csv', index=False)
education_df.to_csv('output/dim_education.csv', index=False)
time_dim.to_csv('output/dim_time.csv', index=False)
industry_dim.to_csv('output/dim_industry.csv', index=False)
career_prog_fact.to_csv('output/fact_career_progression.csv', index=False)


In [22]:
print("=== Data Quality Issues ===")
print(f"Job date inconsistencies: {sum(len(issues) for issues in job_date_issues.values())} issues")
print(f"Expired certifications: {len(expired_certs)} certifications")
print(f"Outliers in professional experience: {len(experience_outliers)} professionals")
print(f"Outliers in skill experience: {len(skill_experience_outliers)} skills")

=== Data Quality Issues ===
Job date inconsistencies: 0 issues
Expired certifications: 19 certifications
Outliers in professional experience: 0 professionals
Outliers in skill experience: 1 skills
